In [1]:
import json
import mysql.connector

In [2]:


mydb = mysql.connector.connect(
  host="database-1.chgxrutzqyvi.eu-central-1.rds.amazonaws.com",
  user="gmaps",
  passwd="oIc1burIsKZYLRxKjZPN",
  database="innodb"
)


mycursor = mydb.cursor()
print(mydb)

# Create tables

In [3]:
# Main table
sql_main_table = """
    CREATE TABLE IF NOT EXISTS commercial_premise (
        id INT NOT NULL AUTO_INCREMENT,
        name VARCHAR(100) NOT NULL,
        zip_code INT(5) NOT NULL,
        type VARCHAR(20) NOT NULL,
        score FLOAT(2) DEFAULT 0.0,
        total_scores INT(10) DEFAULT 0,
        price_range VARCHAR(5),
        style VARCHAR(20),
        address VARCHAR(40) NOT NULL,
        PRIMARY KEY(ID)
    )
"""

sql_comments = """
    CREATE TABLE IF NOT EXISTS commercial_premise_comments (
        id INT NOT NULL AUTO_INCREMENT,
        commercial_premise_id INT NOT NULL,
        content VARCHAR(200),
        PRIMARY KEY(id),
        INDEX prem_ind (commercial_premise_id),
        FOREIGN KEY (commercial_premise_id)
            REFERENCES commercial_premise(id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    )
"""

sql_ocupation = """
    CREATE TABLE IF NOT EXISTS commercial_premise_occupation (
        id INT NOT NULL AUTO_INCREMENT,
        commercial_premise_id INT NOT NULL,
        week_day VARCHAR(9) NOT NULL,
        H00_01 FLOAT(2) DEFAULT 0.0,
        H01_02 FLOAT(2) DEFAULT 0.0,
        H02_03 FLOAT(2) DEFAULT 0.0,
        H03_04 FLOAT(2) DEFAULT 0.0,
        H04_05 FLOAT(2) DEFAULT 0.0,
        H05_06 FLOAT(2) DEFAULT 0.0,
        H06_07 FLOAT(2) DEFAULT 0.0,
        H07_08 FLOAT(2) DEFAULT 0.0,
        H08_09 FLOAT(2) DEFAULT 0.0,
        H09_10 FLOAT(2) DEFAULT 0.0,
        H10_11 FLOAT(2) DEFAULT 0.0,
        H11_12 FLOAT(2) DEFAULT 0.0,
        H12_13 FLOAT(2) DEFAULT 0.0,
        H13_14 FLOAT(2) DEFAULT 0.0,
        H14_15 FLOAT(2) DEFAULT 0.0,
        H15_16 FLOAT(2) DEFAULT 0.0,
        H16_17 FLOAT(2) DEFAULT 0.0,
        H17_18 FLOAT(2) DEFAULT 0.0,
        H18_19 FLOAT(2) DEFAULT 0.0,
        H19_20 FLOAT(2) DEFAULT 0.0,
        H20_21 FLOAT(2) DEFAULT 0.0,
        H21_22 FLOAT(2) DEFAULT 0.0,
        H22_23 FLOAT(2) DEFAULT 0.0,
        H23_24 FLOAT(2) DEFAULT 0.0,
        PRIMARY KEY(id),
        INDEX prem_ind (commercial_premise_id),
        FOREIGN KEY (commercial_premise_id)
            REFERENCES commercial_premise(id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    )
"""
mycursor.execute(sql_main_table)
mycursor.execute(sql_comments)
mycursor.execute(sql_ocupation)
mycursor.execute("SHOW TABLES")
mycursor.fetchall()

[('commercial_premise',),
 ('commercial_premise_comments',),
 ('commercial_premise_occupation',)]

In [4]:
from pprint import pprint
sql1 = "DESCRIBE commercial_premise"
sql2 = "DESCRIBE commercial_premise_comments"
sql3 = "DESCRIBE commercial_premise_occupation"
mycursor.execute(sql1)
pprint(mycursor.fetchall())
mycursor.execute(sql2)
pprint(mycursor.fetchall())
mycursor.execute(sql3)
pprint(mycursor.fetchall())
# sql2 = "DESCRIBE commercial_premise_comments"
# mycursor.execute(sql)
# print(mycursor.fetchall())

[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('name', 'varchar(100)', 'NO', '', None, ''),
 ('zip_code', 'int(5)', 'NO', '', None, ''),
 ('type', 'varchar(20)', 'NO', '', None, ''),
 ('score', 'float', 'YES', '', '0', ''),
 ('total_scores', 'int(10)', 'YES', '', '0', ''),
 ('price_range', 'varchar(5)', 'YES', '', None, ''),
 ('style', 'varchar(20)', 'YES', '', None, ''),
 ('address', 'varchar(40)', 'NO', '', None, '')]
[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('commercial_premise_id', 'int(11)', 'NO', 'MUL', None, ''),
 ('content', 'varchar(200)', 'YES', '', None, '')]
[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('commercial_premise_id', 'int(11)', 'NO', 'MUL', None, ''),
 ('week_day', 'varchar(9)', 'NO', '', None, ''),
 ('H00_01', 'float', 'YES', '', '0', ''),
 ('H01_02', 'float', 'YES', '', '0', ''),
 ('H02_03', 'float', 'YES', '', '0', ''),
 ('H03_04', 'float', 'YES', '', '0', ''),
 ('H04_05', 'float', 'YES', '', '0', ''),
 ('H05_06', 'flo

# Read data

In [4]:
with open("../data.json", "r") as file:
    sample = json.loads(file.read())

In [5]:
sample.keys()

dict_keys(['AHOAN', 'Bar Restaurante Víctor', 'Huevos Y Tortilla', 'con B de bilbao', 'Restaurante Lurrina', 'Restaurante-Cocktail Yamike', 'Café Bar Bilbao'])

# Format data

In [6]:
def decompose_occupancy_data(occupancy_levels):
    occupancy = {
        "lunes": {},
        "martes": {},
        "miercoles": {},
        "jueves": {},
        "viernes": {},
        "sabado": {},
        "domingo": {}
    }
    for week_day, occupancy_levels in occupancy_levels.items():
        if(occupancy_levels is not None):
            for occupancy_level in occupancy_levels:
                if(occupancy_level is not None):
                    try:
                        base = occupancy_level.split(":")[1:]
                        occupancy[week_day].update({
                            base[1].split(")")[0].strip(): float(base[0].split("\xa0%")[0])
                        })
                    except:
                        pass
    return occupancy
        

In [7]:
def write_to_db(element, zip_code, premise_type, mydb, mycursor):
    
    #Store element
    compressed_address = [item.strip() for item in element["address"].split("·")]
    if(len(compressed_address) == 3):
        price_range = compressed_address[0]
        style = compressed_address[1]
        address = compressed_address[2]
    else:
        price_range = None
        style = compressed_address[0]
        address = compressed_address[1]
        
    sql = """
        INSERT INTO commercial_premise 
            (name, zip_code, type, score, total_scores, price_range, style, address) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
    values = (
        element["name"],
        zip_code,
        premise_type,
        element["score"],
        element["total_scores"],
        price_range,
        style,
        address
    )
    mycursor.execute(sql, values)
    mydb.commit()
    element_id = mycursor.lastrowid
    #Store comments
    for comment in element["comments"]:
        sql = """
            INSERT INTO commercial_premise_comments
            (commercial_premise_id, content)
            VALUES (%s, %s)
        """
        values = (
            element_id, comment
        )
        mycursor.execute(sql, values)
        mydb.commit()
    #Store occupancy data
    sql = """
        INSERT INTO commercial_premise_occupation
        (
            commercial_premise_id, week_day,
            H00_01, H01_02, H02_03, H03_04, H04_05, H05_06, H06_07, H07_08, H08_09, H09_10, H10_11, H11_12,
            H12_13, H13_14, H14_15, H15_16, H16_17, H17_18, H18_19, H19_20, H20_21, H21_22, H22_23, H23_24
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """
    for week_day, content in decompose_occupancy_data(element["occupancy"]).items():
        if content is not None and content != {}:
            values = (
                element_id, week_day,
                content["24"], content["01"], content["02"], content["03"], content["04"],
                content["05"], content["06"], content["07"], content["08"], content["09"],
                content["10"], content["11"], content["12"], content["13"], content["14"],
                content["15"], content["16"], content["17"], content["18"], content["19"],
                content["20"], content["21"], content["22"], content["23"]
            )
            mycursor.execute(sql, values)
            mydb.commit()

In [8]:
for name, element in sample.items():
    print(name)
    write_to_db(element, "48005", "restaurante", mydb, mycursor)

AHOAN
Bar Restaurante Víctor
Huevos Y Tortilla
con B de bilbao
Restaurante Lurrina
Restaurante-Cocktail Yamike
Café Bar Bilbao
